# Notes on Python
## Examples for reference, tips, Best Practices

Based on the Course: Core Python (Advanced Generators and Coroutines) at PluralSight

Author: Gonçalo Felício  
Date: 04/2022  
Provided by: ISIWAY

Something like a pocketbook to come to for quick references, examples, and tips of best practices, compiled with my own preferences  
Loosely divided by subject, and with some degree, by the respective modules


### Iterators and Iterables Review
Iterators are objects where I can call `__next__` to return the next item in the sequence  
Iterables are objects that, using the `__iter__` method, return an Iterator  


### Generators
Generators are iterables that are Lazy, Performant and Asynchronous, best way to iterate over a sequence  
Usually will be better performers for less lines of code and readable, as long as the reader understandes generators  
Generators offer a way to suspend execution of a function

Tip: Use 'dict' instead of 'list' with iterables, lookups are faster and memory is much more compact

Generators are great for pipelines as they are lazy, and only actually execute when necessary! This saves a lot of time and memory as nothing is stored, since the execution of the code is decoupled from the definition of the code

In short: Generators produce data for iteration

In [4]:
# nothing is executed until the function is actually called
# this function is a generator factory, because of range, but its not a generator, as it does not contain yield
def pipeline(number):
    data = (i for i in range(number))
    squared = (i**2 for i in data)
    negated = (-i for i in squared)
    return (n + 1 for n in negated)

p = pipeline(10)


In [5]:
list(p)

[1, 0, -3, -8, -15, -24, -35, -48, -63, -80]

Tip: We don't need to always define generators as `itertools` most likely already has all the generators we might require!
Example of all 3 aproaches

In [1]:
import numpy as np
sample = np.random.random(10_000)

In [2]:
def old_style_avg(iterable):
    total_sum = 0
    total_elem = 0
    avg = []
    for number in iterable:
        total_sum += number
        total_elem += 1
        avg.append(total_sum / total_elem)
    return avg

In [3]:
%timeit old_style_avg(sample)

6.78 ms ± 36.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [4]:
%load_ext memory_profiler
%memit old_style_avg(sample)

peak memory: 73.68 MiB, increment: 0.73 MiB


In [5]:
def gen_avg(iterable):
    total_sum = 0
    total_elem = 0
    for number in iterable:
        total_sum += number
        total_elem += 1
        yield total_sum / total_elem

In [6]:
%timeit gen_avg(sample) # much much faster!

263 ns ± 3.1 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [7]:
%memit gen_avg(sample) # same memory 

peak memory: 73.50 MiB, increment: 0.00 MiB


In [8]:
import itertools

In [24]:
itertools.accumulate?

Init signature: itertools.accumulate(iterable, func=None, *, initial=None)
Docstring:      Return series of accumulated sums (or other binary function results).
Type:           type
Subclasses:     


In [26]:
enumerate?

Init signature: enumerate(iterable, start=0)
Docstring:     
Return an enumerate object.

  iterable
    an object supporting iteration

The enumerate object yields pairs containing a count (from start, which
defaults to zero) and a value yielded by the iterable argument.

enumerate is useful for obtaining an indexed list:
    (0, seq[0]), (1, seq[1]), (2, seq[2]), ...
Type:           type
Subclasses:     


In [25]:
itertools.starmap?

Init signature: itertools.starmap(function, iterable, /)
Docstring:      Return an iterator whose values are returned from the function evaluated with an argument tuple taken from the given sequence.
Type:           type
Subclasses:     


In [9]:
def itertool_avg(iterable):
    return itertools.starmap(lambda num_elem, sum_elem: sum_elem / num_elem, 
                             enumerate(itertools.accumulate(iterable), 1))

In [10]:
%timeit itertool_avg(sample) # 3 times slower than the generator

618 ns ± 6.27 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [11]:
%memit itertool_avg(sample) # same memory 

peak memory: 73.55 MiB, increment: 0.00 MiB


Generators can be chained as pipeline to act deferred on a whole dataset  
Itertools offers a lot of precooked generator factories  

### Coroutines
a yield expression yields every value sent to the coroutine and assign that value to the variable on the left  
Good to use a decorator `@coroutine` to avoid colisions for example, forgetting to run the first next that makes a coroutine ready to receive values  

In short: Coroutines are consumers of data

In [89]:
def coroutine(func):
    def start(*args, **kwargs):
        cr = func(*args, **kwargs)
        next(cr)
        return cr
    return start

def my_coroutine(a):
    print(f'Started with {a}')
    b = yield a
    print(f'Recieved {b} to continue! And yielded {a} as well, just because I can')
    yield b

In [53]:
my_coro = my_coroutine(1)

In [54]:
a = next(my_coro)

Started with 1


In [55]:
a

1

In [56]:
b = my_coro.send(2)

Recieved 2 to continue! And yielded 1 as well, just because I can


In [57]:
a,b

(1, 2)

In [58]:
next(my_coro)

StopIteration: 

In [104]:
@coroutine
def coroutine_exception(number):
    print('Coroutine has started')
    while True:
        try:
            x = yield
        except ValueError:
            print('ValueError handled. Continuing...')
        except GeneratorExit:
            print('This is executed if i get closed, final breaths.. *dies*')
            raise # must reraise the exception or caller will not see it
        else:
            print(f'Coroutine recieved:{x}')
            number + x

In [105]:
mycoro = coroutine_exception(1)

Coroutine has started


In [106]:
mycoro.send(2)

Coroutine recieved:2


In [107]:
mycoro.send(3)

Coroutine recieved:3


In [108]:
mycoro.close()

This is executed if i get closed, final breaths.. *dies*


In [109]:
mycoro = coroutine_exception(1)

Coroutine has started


In [110]:
mycoro.throw(ValueError) # can only catch the Error itself, no instances of Error

ValueError handled. Continuing...


In [112]:
mycoro.send(5)

Coroutine recieved:5


In [113]:
mycoro.send(None) # did not catch TypeError so exception is raised and closes gen

Coroutine recieved:None


TypeError: unsupported operand type(s) for +: 'int' and 'NoneType'

In [115]:
next(mycoro)

StopIteration: 

A cool example that uses a bit of everything covered above

In [149]:
from collections import namedtuple
import random
from statistics import mean

@coroutine
def avg_with_result_and_execp():
    print('Coroutine has started')
    Result = namedtuple('Result', ['Count', 'Average'])
    num_elem = 0
    sum_up_to_now = 0
    average = None
    
    while True:
        try:
            value = yield average
        except ValueError:
            print('Value error detected. Continuing...')
        except GeneratorExit:
            print(f'''I have been closed. Dying gracefully.. here's 
                  the final result {Result(num_elem, average)}''')
            raise
        else:
            if value is None:
                break
            num_elem += 1
            sum_up_to_now += value
            average = sum_up_to_now / num_elem

    return Result(num_elem, average)

def main(size):
    averager = avg_with_result_and_execp()
    magic_values = [random.randint(0, 25) for _ in range(size)]
    print(f'Real average is {mean(magic_values)}')
    try:
        for value in magic_values:
            averager.send(value)
            if value == 1:
                averager.throw(ValueError)
            if value == 3:
                averager.close()
        next(averager)
    except StopIteration as e:
        last_average = e.value
        print(f'Final result: {last_average}')
    except GeneratorExit:
        pass

if __name__ == '__main__':
    main(40)

Coroutine has started
Real average is 11.65
I have been closed. Dying gracefully.. here's 
                  the final result Result(Count=24, Average=12.5)
Final result: None


The coroutine above runs for a 'size' number of times with random integers between 0 and 25 and if '1' appears, it throws and handles it as a ValueError. If '3' appears it closes the coroutine. If no exception is raised it ends the coroutine, catches the StopIteration exception and returns the final result from the exception value!

Coroutines are capable of receiving and returning data!

Run a few times and increase 'size' to cause the GeneratorExit, ValueError and StopIteration exceptions to see the different behaviours

### Sub generators
`Yield from` enables to delegate work to a sub generator via a pipe  
The caller can yield, send or throw to the reference of the sub generator  
The return statement in the `yield from` is magically handled and sent an expression

Coroutines are like tasks, so it's possible to send tasks and receive the work back